In [1]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from joblib import Parallel, delayed
import pickle
import re
import multiprocessing



In [4]:
search1_all= pd.read_excel('data/search1_all_alternative.xlsx')
link = pd.read_csv('gvkey_cik_link.csv', encoding='latin1')
link_permno = pd.read_csv('cik_permno.csv', encoding='latin1')

C:\Users\TJR\AppData\Local\Temp\ipykernel_9168\2737679023.py:3: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  link_permno = pd.read_csv('cik_permno.csv', encoding='latin1')


In [3]:
# use sequence as unique identifier
search1_all['unique_id'] = search1_all.index

df = pd.merge(search1_all, link, on='cik', how='left')

# create a new column, identify whether date is between link_start_date and link_end_date
df['date'] = pd.to_datetime(df['date'])
df['link_start_date'] = pd.to_datetime(df['link_start_date'], format='%Y-%m-%d', errors='coerce')
df['link_end_date'] = pd.to_datetime(df['link_end_date'], format='%Y-%m-%d', errors='coerce')
df['link_date'] = (df['date'] >= df['link_start_date']) & (df['date'] <= df['link_end_date'])

# for loop each unique id, 1. if there is more than one row, keep the ones with link_date = True 2. if there is only one row, keep it
def filter_link_date(df):
    if len(df) > 1 and df['link_date'].sum() > 0:
        return df[df['link_date']]
    else:
        return df

df = df.groupby('unique_id').apply(filter_link_date).reset_index(drop=True)
print(df.unique_id.nunique())

df.drop_duplicates(subset=['unique_id'], keep='first', inplace=True)
for year in range(2000,2021):
    df_year = df[df['year'] == year].drop(columns=['unique_id', 'link_start_date', 'link_end_date', 'link_date'])
    df_year.to_csv(f'search1_{year}.csv', index=False)

,Unnamed: 0,form,date,cik,company,link,doc,keyword,serp,year,unique_id,gvkey,source,link_desc,sec_company_name,link_company_name,link_start_date,link_end_date,link_date
0,0001193125-20-013900,DEF 14A,2020-01-24,70145,NATIONAL FUEL GAS CO,https://www.sec.gov/Archives/edgar/data/70145/...,d) | All management participants who were hire...,['benefit plan that was frozen in 1987'],0,2020,0,7733.0,CRSP/Compustat Merged,Valid CIK-GVKEY Link,NATIONAL FUEL GAS CO,NATIONAL FUEL GAS CO,2007-04-14,2023-12-08,True
3,0001308179-20-000279,DEF 14A,2020-10-07,96021,SYSCO CORP,https://www.sec.gov/Archives/edgar/data/96021/...,• | The accrued benefit determined as of June ...,"['benefit determined as of June 29, 2013 (the ...",1,2020,1,10247.0,CRSP/Compustat Merged,Valid CIK-GVKEY Link,SYSCO CORP,SYSCO CORP,2007-04-14,2023-09-08,True
5,0001558370-20-011349,10-K,2020-10-01,1751700,"TEB Bancorp, Inc.",https://www.sec.gov/Archives/edgar/data/175170...,Defined Benefit Pension Plan \nThe Bank has a ...,['defined benefit payments upon retirement or ...,0,2020,2,35167.0,CRSP/Compustat Merged,Valid CIK-GVKEY Link,"TEB BANCORP, INC.",TEB BANCORP INC,2020-09-12,2024-01-12,True
6,0001206774-20-002817,DEF 14A,2020-10-02,16732,CAMPBELL SOUP CO,https://www.sec.gov/Archives/edgar/data/16732/...,The Qualified Plan\n The Qualified Plan was es...,['1999 (including Mr. Foley) are instead eligi...,0,2020,3,2663.0,CRSP/Compustat Merged,Valid CIK-GVKEY Link,CAMPBELL SOUP CO,CAMPBELL SOUP CO,2007-04-14,2022-10-07,True
8,0001140361-20-022426,DEF 14A,2020-10-05,1564708,NEWS CORP,https://www.sec.gov/Archives/edgar/data/156470...,EXECUTIVE COMPENSATION\n\n\nSupplementary Plan...,['benefit is payable as a single lump sum upon...,0,2020,4,18043.0,CRSP/Compustat Merged,Valid CIK-GVKEY Link,NEWS CORP,NEWS CORP,2013-07-15,2023-10-13,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17694,0000067347-03-000075,10-K/A,2003-02-12,67347,MODINE MANUFACTURING CO,https://www.sec.gov/Archives/edgar/data/67347/...,4(b)(i) | Rights Agreement Amendment No. 1 dat...,"['Retirement Plan (effective April 1, 1992 and...",0,2003,6929,7486.0,Compustat Company,Valid CIK-GVKEY Link,MODINE MANUFACTURING CO,MODINE MANUFACTURING CO,1963-03-31,2024-03-31,True
17695,0001047469-03-004776,10-K/A,2003-02-11,1042801,STATEN ISLAND BANCORP INC,https://www.sec.gov/Archives/edgar/data/104280...,The Bank amended the defined benefit plan to f...,"[""defined benefit plan to freeze future benefi...",0,2003,6930,66096.0,Compustat Company,Valid CIK-GVKEY Link,STATEN ISLAND BANCORP INC,STATEN ISLAND BANCORP INC,1996-12-31,2003-12-31,True
17697,0001047469-03-004776,10-K/A,2003-02-11,1042801,STATEN ISLAND BANCORP INC,https://www.sec.gov/Archives/edgar/data/104280...,SUPPLEMENTAL EXECUTIVE RETIREMENT PLAN. In 199...,"['retirement, death or disability. The Company...",0,2003,6931,66096.0,Compustat Company,Valid CIK-GVKEY Link,STATEN ISLAND BANCORP INC,STATEN ISLAND BANCORP INC,1996-12-31,2003-12-31,True
17699,0000930661-03-000515,10-Q,2003-02-11,884124,GALEY & LORD INC,https://www.sec.gov/Archives/edgar/data/884124...,"On September 20, 2001, the Company froze the a...","['2001, the Company froze the accrual of futur...",0,2003,6932,25202.0,Compustat Company,Valid CIK-GVKEY Link,GALEY & LORD INC,GALEY & LORD INC,1991-09-30,2003-09-30,True
